### Data preprocessing

In [3]:
file_path = "SemEval2018-T3-train-taskA.txt"

In [4]:
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re

fo = open(file_path,encoding="utf8")
trainingSet = fo.readlines()

In [5]:
labels = []
tweets = []

#fetch tweets and labels (sarcasm or no sarcasm)
for line in trainingSet:
    if not line.lower().startswith("tweet index"):
        line = line.rstrip()
        splitLine = line.split("\t")
        labels.append(splitLine[1])
        tweets.append(splitLine[2])
        
#remove email and @references
for idx, tweet in enumerate(tweets):
    newTweet = re.sub('(http.+(\s)?)|((@\w*\d*(\s)?))','',tweet,flags=re.MULTILINE)
    tweets[idx] = newTweet

### 1. Unigram feature with vectorization of tweets

In [6]:
def vectorize_bow(tweets):
    tweet_tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True).tokenize
    tweet_vectorizer = TfidfVectorizer(
        strip_accents="unicode", analyzer="word", tokenizer=tweet_tokenizer, stop_words="english",max_features = 3000)
    vector = tweet_vectorizer.fit_transform(tweets)
    vector = vector.toarray()
    return vector

In [7]:
import numpy as np
tweet_vector = vectorize_bow(tweets)
fo = open("unigram_vector.npy","wb")
np.save(fo, tweet_vector)
fo.close()

In [8]:
print(tweet_vector)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.17412277 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.15014354 0.         0.         ... 0.         0.         0.        ]]


### 2. Punctuation feature

In [7]:
def print_to_file(data, filename,column):
    fo = open(filename+".txt","w+")
    fo.write(column)
    fo.write("\n")
    for line in data:
        fo.write(line)
        fo.write("\n")
    fo.close()

In [8]:
punc = []
ellipsis = []

for tweet in tweets:
    exclaim_count = tweet.count("!")
    question_count = tweet.count("?")
    comma_count = tweet.count(",")
    total_count = exclaim_count + question_count + comma_count
    
    ellipsis_count = len(re.findall('[(\.\.\.)]+',tweet))
    
    punc.append(str(total_count)+"\t"+str(ellipsis_count))
print_to_file(punc,"punctuation","punctuation_count" + "\t" + "ellipsis_count")

In [9]:
fo = open("punctuations.npy","wb")
np.save(fo, total_count)
fo.close()

In [10]:
fo = open("ellipsis.npy","wb")
np.save(fo, ellipsis_count)
fo.close()

In [12]:
print_to_file(labels,"labels","labels")